# DIVAnd analysis using the sample data set

This example performs a salinity analysis using data from the Black Sea.     
The analysis is done for every season and year (using all data with 10-year sliding windows form the same season).

For testing purposes, let's start with a low resolution.

A slightly large test case:
* horizontal resolution of 0.1 degree for the Black Sea
* 51 depths levels
* 8 time instance
* fixed correlation length
* CPU time: 21 minutes
* CPU time increases linearly with the number of time instance.

In [ ]:
using DIVAnd
using PyPlot
using NCDatasets
using DataStructures
using Dates
using Statistics
include("../config.jl")

Download the file `Salinity.bigfile` (if it is not already present). It will be placed in the same directory as this notebook. This file based on the World Ocean Database.

In [ ]:
fname = salinitybigfile
download_check(fname, salinitybigfileURL)

Download the bathymetry if it is not already present

In [ ]:
bathname = gebco16file
download_check(gebco16file, gebco16fileURL)
bathisglobal = true

Load the data which is here a simple text file with the following columns

| longitude | latitude | value | depth | flag | year | month | day | hour | time string | identifier | bottom depth |
|----------:|---------:|------:|------:|-------:|------:|-----:|-----|------:|-------:|-------:|-------:|
| 28.3333 | 43.167 | 15.7020 | 0.0 | 0 | 1991 | 09 | 03 | 16 | 1991-09-03T16:25 | Cruise:WOD05_BG000003-11570900-28.3333-43.167 | 36 | 
| 28.3333 | 43.167 | 15.7380 | 10.0 | 0 | 1991 | 09 | 03 | 16 | 1991-09-03T16:25 | Cruise:WOD05_BG000003-11570900-28.3333-43.167 | 36 |

Only longitude, latitude, depth, time and the values are used.

In [ ]:
obsval,obslon,obslat,obsdepth,obstime,obsid = loadbigfile(fname);
@show size(obsval);

In [ ]:
replace(split(obsid[1], ":")[2], "_"=>"-")

In [ ]:
obsid_mod = [join((split(replace(split(oo, ":")[2], "_"=>"-"), "-")[1:3]), "-") for oo in obsid];

The file contains about 140000 measurements.     
Print some basic statistics about the data. Keep an eye on the ranges.

In [ ]:
checkobs((obslon,obslat,obsdepth,obstime),obsval,obsid)

Define the resolution. It is recommended to start with a low resolution and increase the resolution when the results are suitable.

In [ ]:
dx = 0.2 # longitude resolution in degrees
dy = 0.2 # latitude resolution in degrees

Define the bounding box of the spatial domain

In [ ]:
lonr = 27.:dx:42.; # the range of longitudes (start:step:end)
latr = 40.:dy:47.; # the range of latitudes (start:step:end)

Define the depth levels

In [ ]:
depthr = [0.,5., 10., 15., 20., 25., 30., 40., 50., 66, 
    75, 85, 100, 112, 125, 135, 150, 175, 200, 225, 250, 
    275, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 
    800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 
    1300, 1350, 1400, 1450, 1500, 1600, 1750, 1850, 2000]

# or for testing just a few levels
depthr = [0.,20.,50.]

@show size(depthr);

The error variance of the observation (scaled by the error variance of the background).    
Note that this is the inverse of the signal-to-noise ratio used in the 2D version of `DIVA`.

In [ ]:
epsilon2 = 0.01;

* Correlation length in meters (in x, y, and z directions)
* 200_000. is the same as 200000., but the former is easier to read.

In [ ]:
sz = (length(lonr),length(latr),length(depthr))

lenx = fill(200_000.,sz)
leny = fill(200_000.,sz)
lenz = [10+depthr[k]/15 for i = 1:sz[1], j = 1:sz[2], k = 1:sz[3]];

Year range and averaging time window

In [ ]:
years = 1993:1994

year_window = 10

# winter: January-March    1,2,3
# spring: April-June       4,5,6
# summer: July-September   7,8,9
# autumn: October-December 10,11,12

monthlists = [
    [1,2,3],
    [4,5,6],
    [7,8,9],
    [10,11,12]
];

* Other possible choises for the time selector are `TimeSelectorYearListMonthList` and `TimeSelectorRunningAverage`.
* Type `?` followed by these functions

In [ ]:
?TimeSelectorYW

In [ ]:
TS = TimeSelectorYW(years,year_window,monthlists)

varname = "Salinity"

# File name based on the variable (but all spaces are replaced by _)
filename = "Water_body_$(replace(varname, " "=>"_")).4Danl.nc"

# Time origin for the NetCDF file
timeorigin = DateTime(1900,1,1,0,0,0)

# Extract the bathymetry for plotting
bx,by,b = extract_bath(bathname,bathisglobal,lonr,latr);

A list of all metadata

In [ ]:
metadata = OrderedDict(
    # Name of the project (SeaDataCloud, SeaDataNet, EMODNET-chemistry, ...)
    "project" => "SeaDataCloud",

    # URN code for the institution EDMO registry,
    # e.g. SDN:EDMO::1579
    "institution_urn" => "SDN:EDMO::1579",

    # Production group
    #"production" => "Diva group",

    # Name and emails from authors
    "Author_e-mail" => ["Your Name1 <name1@example.com>", "Other Name <name2@example.com>"],

    # Source of the observation
    "source" => "observational data from SeaDataNet/EMODNet Chemistry Data Network",

    # Additional comment
    "comment" => "...",

    # SeaDataNet Vocabulary P35 URN
    # http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=p35
    # example: SDN:P35::WATERTEMP
    "parameter_keyword_urn" => "SDN:P35::EPC00001",

    # List of SeaDataNet Parameter Discovery Vocabulary P02 URNs
    # http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=p02
    # example: ["SDN:P02::TEMP"]
    "search_keywords_urn" => ["SDN:P02::PSAL"],

    # List of SeaDataNet Vocabulary C19 area URNs
    # SeaVoX salt and fresh water body gazetteer (C19)
    # http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=C19
    # example: ["SDN:C19::3_1"]
    "area_keywords_urn" => ["SDN:C19::3_3"],

    "product_version" => "1.0",
    
    "product_code" => "something-to-decide",
    
    # bathymetry source acknowledgement
    # see, e.g.
    # * EMODnet Bathymetry Consortium (2016): EMODnet Digital Bathymetry (DTM).
    # https://doi.org/10.12770/c7b53704-999d-4721-b1a3-04ec60c87238
    # 
    # taken from
    # http://www.emodnet-bathymetry.eu/data-products/acknowledgement-in-publications
    #
    # * The GEBCO Digital Atlas published by the British Oceanographic Data Centre on behalf of IOC and IHO, 2003
    #
    # taken from
    # https://www.bodc.ac.uk/projects/data_management/international/gebco/gebco_digital_atlas/copyright_and_attribution/
        
    "bathymetry_source" => "The GEBCO Digital Atlas published by the British Oceanographic Data Centre on behalf of IOC and IHO, 2003",

    # NetCDF CF standard name
    # http://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html
    # example "standard_name" = "sea_water_temperature",
    "netcdf_standard_name" => "sea_water_salinity",

    "netcdf_long_name" => "sea water salinity",

    "netcdf_units" => "1e-3",

    # Abstract for the product
    "abstract" => "...",

    # This option provides a place to acknowledge various types of support for the
    # project that produced the data
    "acknowledgement" => "...",

    "documentation" => "https://doi.org/doi_of_doc",

    # Digital Object Identifier of the data product
    "doi" => "...")

* Make the NetCDF global and variable attributes based on the metadata.
* Custom attributes can be added by changing `ncglobalattrib`:

```julia
ncglobalattrib["attribute_name"] = "attribute_value"
```


In [ ]:
ncglobalattrib, ncvarattrib = SDNMetadata(metadata,filename,varname,lonr,latr)

Plot the results near the surface for debugging and quick inspection.

To generate the plots, we define a function `plotres` (see next cell) that will be used as an optional argument when we call the interpolation with `DIVAnd`.

In [ ]:
function plotres(timeindex,sel,fit,erri)
    tmp = copy(fit)
    tmp[erri .> .5] .= NaN;
    figure(figsize = (10,5))

    # select the data near the surface
    selsurface = sel .& (obsdepth .< 5)
    vmin,vmax = quantile(obsval[selsurface],(0.01,0.99))

    # plot the data
    subplot(1,2,1)
    scatter(obslon[selsurface],obslat[selsurface],10,obsval[selsurface];
            vmin = vmin, vmax = vmax)
    xlim(minimum(lonr),maximum(lonr))
    ylim(minimum(latr),maximum(latr))
    colorbar(orientation="horizontal")
    contourf(bx,by,b', levels = [-1e5,0],colors = [[.5,.5,.5]])
    gca().set_aspect(1/cosd(mean(by)))
    title("Observations ($(timeindex))")

    # plot the analysis
    subplot(1,2,2)
    pcolor(lonr,latr,permutedims(tmp[:,:,1],[2,1]);
           vmin = vmin, vmax = vmax)
    colorbar(orientation="horizontal")
    contourf(bx,by,b', levels = [-1e5,0],colors = [[.5,.5,.5]])
    gca().set_aspect(1/cosd(mean(by)))
    title("Analysis ($(timeindex))")
end

If you do not want to generate plots but print the time index at every time slice
you can use the function `plotres_timeindex`.

In [ ]:
function plotres_timeindex(timeindex,sel,fit,erri)
    @show timeindex
end

Launch the analysis. Plotting can be disabled by commenting the line containing `plotres`.

Here we use `diva3d`, which performs a series of 3D analyses (lon, lat and depth) for the different periods defined previously.

In [ ]:
if isfile(filename)
   rm(filename) # delete the previous analysis
end

@time dbinfo = diva3d((lonr,latr,depthr,TS),
              (obslon,obslat,obsdepth,obstime),
              obsval,
              (lenx,leny,lenz),
              epsilon2,
              filename,varname,
              bathname = bathname,
              bathisglobal = bathisglobal,
              plotres = plotres,
              ncvarattrib = ncvarattrib,
              ncglobalattrib = ncglobalattrib,
              timeorigin = timeorigin
       );


residual = dbinfo[:residuals];

This generates many plots (one for every time instance).     
The number in the title is the time index starting with the first season and year.


Save the observation metadata in the NetCDF file

In [ ]:
DIVAnd.saveobs(filename,(obslon,obslat,obsdepth,obstime),obsid_mod)

# Residuals

Range of the residuals

In [ ]:
res = get(dbinfo, :residuals, 0)
@show extrema(res);

Residuals with NaNs removed

In [ ]:
res = residual[.!isnan.(residual)]

@show extrema(res);
@show quantile(res,[0.01,0.99]);

Get the identifier of the anomalous point

In [ ]:
i = findfirst(minimum(res) .== residual)
obsid[i]

For future analysis the residuals can also be saved.

In [ ]:
outputdir = "./output/"
if !isdir(outputdir)
    mkpath(outputdir)
end

resname = joinpath(outputdir, "Water_body_$(replace(varname, " "=>"_")).residuals.nc")
if isfile(resname)
   rm(resname)
end
DIVAnd.saveobs(resname,"$(varname)_residual",residual,(obslon,obslat,obsdepth,obstime),obsid)

* Visualize the residuals (observations minus analysis)
* Change depth and time
* Adjust colorbare range (`clim`) to see large residuals

In [ ]:
sel = (obsdepth .<  50) .& (Dates.month.(obstime) .== 1)
print("Number of data points: $(sum(sel))")
contourf(bx,by,b', levels = [-1e5,0],colors = [[.5,.5,.5]])
scatter(obslon[sel],obslat[sel],10,residual[sel]; cmap="RdBu_r")
clim(-1,1)
# set the correct aspect ratio
gca().set_aspect(1/cosd(mean(latr)))
colorbar(orientation = "horizontal");